## Import libraries and load CSVs

In [47]:
import pandas as pd
import numpy as np
import datetime
import math

#read in Lab with only columns of interest
Lab = pd.read_csv("LabReduced.csv", usecols = ['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc', 'UnitOfMeasure_calc'])
#read in Exam with only columns of interest
Exam = pd.read_csv("Exam.csv", usecols = ['Patient_ID', 'Exam1', 'Result1_calc', 'Exam2', 'Result2_calc', 'UnitOfMeasure_calc', 'DateCreated'])
#read in DiseaseCase with only columns of interest
DiseaseCase = pd.read_csv("DiseaseCase.csv", usecols = ['Patient_ID', 'DateOfOnset'])
#read in Billing

C:\Users\niche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## List all types of lab and exam tests and remove rows with null result_calc values

In [48]:
Lab = Lab.dropna()
pd.set_option('display.max_rows', 100)
print(Lab['Name_calc'].value_counts(ascending=False))

Creatinine (Serum-Cr)                                                                           1621753
Hemoglobin A1c (HbA1c)                                                                          1461598
Hemoglobin (Hb)                                                                                 1418319
Erythrocyte Mean Corpuscular Volume (MCV)                                                       1247198
Leukocytes in Blood (WBC)                                                                       1156710
Erythrocyte Mean Corpuscular Hemoglobin Concentration (MCHC)                                    1154266
Platelets in Blood (PLT)                                                                        1142699
Plasma Creatinine Clearance (eGFR)                                                              1114447
Erythrocyte Distribution Width [Ratio] (RDW_CV)                                                 1104308
Erythrocytes in Blood (RBC)                                     

#### Restructure exam sheet to have one result per row

In [49]:
Exam1 = Exam.drop(columns=['Exam2','Result2_calc']).rename(columns={'Exam1':'Name_calc','Result1_calc':'TestResult_calc'})
Exam2 = Exam.drop(columns=['Exam1','Result1_calc']).rename(columns={'Exam2':'Name_calc','Result2_calc':'TestResult_calc'})
Exam = pd.concat([Exam1,Exam2]).dropna()
print(Exam['Name_calc'].value_counts(ascending=False))

sBP (mmHg)                  2570192
dBP (mmHg)                  2570192
Weight (kg)                 1102589
BMI (kg/m^2)                 910459
Height (cm)                  736942
Waist Circumference (cm)     115353
PEFR (L/min)                     35
Name: Name_calc, dtype: int64


## Filter lab and exam results for only relevant types

In [50]:
#filter lab results for one of 17 relevant lab results, 
validTests = ['Alanine Aminotransferase in Serum or Plasma (ALT)',
              'Aspartate Aminotransferase in Serum or Plasma (AST)',
              'Urea (mM)',
              'Plasma Creatinine Clearance (eGFR)',
              'Potassium (mM)',
              'Sodium (mM)',
              'Gamma Glutamyl Transferase in Serum or Plasma (GGT)',
              'Leukocytes in Blood (WBC)',
              'Protein, total (g/L)',
              'Total Cholesterol (TCh)',
              'Total Bilirubin (TBIL)',
              'Platelets in Blood (PLT)',
              'Fasting Blood Glucose (FBG)',
              'Hemoglobin (Hb)',
              'Erythrocytes in Blood (RBC)',
              'Albumin (g/L)',
              'Hemoglobin A1c (HbA1c)',
              'BMI (kg/m^2)']
Lab = Lab.loc[Lab.Name_calc.isin(validTests)]
Exam = Exam.loc[Exam.Name_calc.isin(validTests)]

#### Merge chosen lab and exam results into combined Results df

In [51]:
#reorder and rename columns before merge
Exam = Exam.rename(columns={'DateCreated':'PerformedDate'})
Exam = Exam[Exam.columns[[0,4,1,2,3]]]

In [53]:
Results = pd.concat([Lab,Exam])

## Filter results to only include tests done after diabetes diagnosis

In [58]:
#order Diabetes patients results and only keep earliest diagnosis
DiseaseCase['DateOfOnset'] = pd.to_datetime(DiseaseCase.DateOfOnset,format='%Y-%m-%d')
DiseaseCase = DiseaseCase.sort_values(by='DateOfOnset')
#only keep earliest diagnosis
DiseaseCase = DiseaseCase.drop_duplicates(['Patient_ID'])
#reassign series of diabetes patient IDs
DiseaseCase.head()

,Patient_ID,DateOfOnset
126536,8080013258526,1980-01-01
251187,8080013270666,1980-01-01
233142,8080613254084,1980-01-01
126857,8080013256071,1980-01-01
126524,8080013215551,1980-01-01


In [69]:
#ensure data column is in correct format
Results.PerformedDate = pd.to_datetime(Results.PerformedDate,format='%Y-%m-%d')
#merge Results and DiseaseCase dfs for date comparison
Results = pd.merge(Results, DiseaseCase, on='Patient_ID')

In [71]:
Results = Results.loc[Results.PerformedDate > Results.DateOfOnset]

#### make test result values numerical, drop rows with null entries

In [75]:
Results.TestResult_calc = pd.to_numeric(Results.TestResult_calc,errors='coerce')
Results = Results.dropna()

### Check that all units are consistent in test values

In [76]:
for test in validTests:
    units = Results.loc[Results.Name_calc == test].UnitOfMeasure_calc.unique()
    print(test, ' - ', units)

Alanine Aminotransferase in Serum or Plasma (ALT)  -  ['U/L']
Aspartate Aminotransferase in Serum or Plasma (AST)  -  ['U/L']
Urea (mM)  -  ['mmol/L']
Plasma Creatinine Clearance (eGFR)  -  ['mL/min']
Potassium (mM)  -  ['mmol/L']
Sodium (mM)  -  ['mmol/L']
Gamma Glutamyl Transferase in Serum or Plasma (GGT)  -  ['U/L']
Leukocytes in Blood (WBC)  -  ['10^9/L']
Protein, total (g/L)  -  ['g/L']
Total Cholesterol (TCh)  -  ['mmol/L']
Total Bilirubin (TBIL)  -  ['umol/L' 'mg/L']
Platelets in Blood (PLT)  -  ['10^9/L']
Fasting Blood Glucose (FBG)  -  ['mmol/L']
Hemoglobin (Hb)  -  ['g/L' 'mmol/L']
Erythrocytes in Blood (RBC)  -  ['10^12/L']
Albumin (g/L)  -  ['g/L']
Hemoglobin A1c (HbA1c)  -  ['%']
BMI (kg/m^2)  -  ['kg/m^2']


In [84]:
#remove vast minority units (mg/L for Total Bilirubin (n = 40) and mmol/L for Hemoglobin (n = 10))
badIndices = Results.loc[(Results.Name_calc == 'Total Bilirubin (TBIL)') & (Results.UnitOfMeasure_calc == 'mg/L')].index
badIndices2 = Results.loc[(Results.Name_calc == 'Hemoglobin (Hb)') & (Results.UnitOfMeasure_calc == 'mmol/L')].index
Results = Results.drop(badIndices)
Results = Results.drop(badIndices2)

## Crop data for each result type within range [q1 - IQR, q3+IQR]

In [90]:
for test in validTests:
    q1 = Results.loc[Results.Name_calc == test].TestResult_calc.quantile(.25)
    q3 = Results.loc[Results.Name_calc == test].TestResult_calc.quantile(.75)
    rng = q3 - q1
    badIndices = Results.loc[(Results.Name_calc == test) & ((Results.TestResult_calc < (q1 - rng)) 
                                    |(Results.TestResult_calc > (q3 + rng)))].index
    Results = Results.drop(badIndices)

## Create dataframe of turtile ranges

In [103]:
names = pd.Series(validTests)
turtile1 = []
turtile2 = []
for test in validTests:
    turtile1.append(Results.loc[Results.Name_calc == test].TestResult_calc.quantile(0.3333))
    turtile2.append(Results.loc[Results.Name_calc == test].TestResult_calc.quantile(0.6667))
t1 = pd.Series(turtile1)
t2 = pd.Series(turtile2)
TestTurtiles = pd.DataFrame(data = {'TestName' : names, 't1' : t1, 't2' : t2})